In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

c:\Users\a882764\.anaconda\envs\Medical\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PINECONE_API_KEY='pcsk_27aGrV_HEbGLTexwW5iE3PDczmP19XqdhtLT4FqVU5oymbFjuBXidW2fNqESX7ktNn69JH'

In [3]:
pc = Pinecone(api_key='PINECONE_API_KEY')

In [ ]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 6973


In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

C:\Users\a882764\AppData\Local\Temp\ipykernel_18208\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [12]:
import os

# ✅ Set API key as environment variable (best practice)
os.environ['PINECONE_API_KEY'] = "pcsk_27aGrV_HEbGLTexwW5iE3PDczmP19XqdhtLT4FqVU5oymbFjuBXidW2fNqESX7ktNn69JH"

# ✅ Initialize Pinecone
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

In [13]:
index_name = "medical"
texts = [t.page_content for t in text_chunks]

docsearch = PineconeVectorStore.from_texts(texts, embeddings, index_name=index_name)


In [14]:

index_name = "medical"
dosearch = PineconeVectorStore.from_existing_index(index_name, embeddings)

In [15]:
dosearch

In [16]:
#If we already have an index we can load it like this
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(id='f46a6547-42fe-4b55-a0ad-779ea152395e', metadata={}, page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.\nKEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction—An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runny nose.\nConjunctiva—The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.'), Document(id='96fe4a28-65b7-4efa-99aa-4cb960796b34', metadata={}, page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.\nKEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction—An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy wat

In [17]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [18]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
chain_type_kwargs = {"prompt": PROMPT}

In [19]:
llm = CTransformers(model="model/llama-2-13b-chat.ggmlv3.q5_1.bin",
                    model_type="llama",
                    config={'max_new_tokens': 512, 'temperature': 0.8})

In [20]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                chain_type="stuff",
                                retriever=dosearch.as_retriever(search_kwargs={'k':2}),
                                return_source_documents=True,
                                chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])